<a href="https://colab.research.google.com/github/sagiodev/FramePack_Colab/blob/main/FramePack_colab_SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [stable-diffusion-art.com](https://stable-diffusion-art.com) - Check out latest tutorials.

### [Guide to use this notebook](https://stable-diffusion-art.com/framepack-colab/) - Leave comment if you have questions.


#### <b>Become a member to support my work.</b>

[<img src="https://stable-diffusion-art.com/wp-content/uploads/2023/10/see_member_benefit_button300.png" width=200>](https://stable-diffusion-art.com/member)


This notebook launches [FramePack](https://github.com/lllyasviel/FramePack).

## Update
- 5/02/2025: Added ngrok support.
- 4/29/2025: First version.


In [ ]:
#@title <font size="6" color="orange">FramePack</font>
#@markdown #### <--- Click to run. Follow the **public** URL link **`https://xxxxxxx.gradio.live`** to launch the app.
NGROK='' #@param {type: "string"}
#@markdown Use NGROK if Gradio has connection issues. Get API key [here](https://dashboard.ngrok.com/get-started/your-authtoken). You can also use [notebook secret](https://stable-diffusion-art.com/comfyui-colab/#Notebook_secrets): `NGROK`.

# Save_Videos_in_Google_Drive = True #@param{type: "boolean"}
# Save_folder = 'AI_PICS/FramePack' #@param {type:"string"}

def ngrok_thread(port):
  PASSWORD='stablediffusionart'
  import time
  import socket
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  ngrok.set_auth_token(NGROK)
  http_tunnel = ngrok.connect(port, auth=f"{PASSWORD}:{PASSWORD}")
  print(f'Username/password: {PASSWORD}')
  print(f'ngrok public URL: {http_tunnel.public_url}')

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Python 3.10 detected. Skip installation.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def clear():
    from IPython.display import clear_output; return clear_output()


# Get secrets
from google.colab import userdata
try:
  NGROK = userdata.get('NGROK')
  print('Used NGROK secret.')
except:
  pass

# ngrok
if NGROK:
  %pip install pyngrok
  from pyngrok import ngrok

# save in google drive
# if Save_Videos_in_Google_Drive:
#   import os
#   from google.colab import drive
#   drive.mount('/content/drive')

# install python 3.10
updatePython()

# install FramePack
%cd /content/
!git clone https://github.com/lllyasviel/FramePack
%cd FramePack
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install -r requirements.txt
!pip install sageattention==1.0.6
# if Save_Videos_in_Google_Drive:
#     output_path = '/content/drive/MyDrive/' + Save_folder
#     if not os.path.exists(output_path):
#       %mkdir -p {output_path}
#     !rm -rf /content/FramePack/outputs/
#     !ln -s {output_path} /content/FramePack/outputs



# Run
clear()
if NGROK:
  import threading
  # ngrok
  port = 7860
  # if not 'ngrok_listener' in locals():
  #   ngrok_listener = ngrok_connect(port)
  threading.Thread(target=ngrok_thread, daemon=True, args=(port,)).start()
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True python demo_gradio.py --share